In [1]:
%matplotlib widget
from astropy.table import Table
from tqdm.notebook import tqdm
from types import SimpleNamespace
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
import shutil
import subprocess
import itertools

In [2]:
sys.path.append('/home/aalvarez/Work/Projects/Elements/5.8/InstallArea/x86_64-fc31-gcc92-dbg/python/')
sys.path.append('/home/aalvarez/Work/Projects/PHZ_NNPZ/PHZ_NNPZ/NNPZ/python/')
os.environ['ELEMENTS_AUX_PATH'] = '/home/aalvarez/Work/Projects/PHZ_NNPZ/PHZ_NNPZ/NNPZ/auxdir/'
os.environ['PATH'] = os.environ['PATH'] + ':/home/aalvarez/Work/Projects/PHZ_NNPZ/PHZ_NNPZ/build.x86_64-fc31-gcc92-dbg/scripts/'

In [3]:
from nnpz.photometry import *
from nnpz.reference_sample.ReferenceSample import ReferenceSample

In [4]:
filter_provider = DirectoryFilterProvider('/home/aalvarez/Work/Data/WillHartley/DC3')

# Filter / SED integration
As SEDs and filters can, and do, have different binning, when integrating a SED through a filter, NNPZ <= 0.6 truncates the SED to the covered range, and interpolates the filter into the SED binning. The code is as follows:

```python
# Truncate the SED to the size of the filter
trunc_sed = self.__truncateSed(sed, self.__filter_range_map[filter_name])

# Interpolate the filter to the SED knots
interp_filter = np.interp(trunc_sed[:,0], filter_trans[:,0], filter_trans[:,1], left=0, right=0)

# Compute the SED through the filter
filtered_sed = trunc_sed[:,1] * interp_filter

# Compute the intensity of the filtered object
intensity = np.trapz(filtered_sed, x=trunc_sed[:,0])
```

**Hypothesis:** Previous checks showed that the ratio between Will's integration and NNPZ integration depends on filter, SED _and_ redshift.
When applying the redshift to a SED, an effect is an spacing on the binning. If we interpolate the filter into the SED grid we may lose marked features
of the filter.

## Verify that indeed the bin size depends on the redshift

In [5]:
def truncateSed(sed, range):
    """Truncates the given SED at the given range"""
    min_i = np.searchsorted(sed[:, 0], range[0])
    if min_i > 0:
        min_i -= 1
    max_i = np.searchsorted(sed[:, 0], range[1])
    max_i += 1
    return sed[min_i:max_i+1, :]

In [6]:
ref_catalog = Table.read('/home/aalvarez/Work/Data/WillHartley/ReferenceSubset.fits')
np.max(ref_catalog['z'])

2.9787273

In [7]:
sed_names = np.unique(ref_catalog['SED_name'])

In [8]:
ref = ReferenceSample('/home/aalvarez/Work/Data/WillHartley/ReferenceSample/')

ValueError: buffer is smaller than requested size

In [ ]:
def get_representative(z, sed):
    shiftf = np.logical_and(ref_catalog['z'] >= z - 0.1, ref_catalog['z'] < z + 0.1)
    sedf = ref_catalog['SED_name'] == sed
    return np.random.choice(ref_catalog[np.logical_and(shiftf, sedf)])

In [ ]:
def plot_by_sed(sed_pick):
    grid = np.arange(100., 21000.,1.)
    
    redshifted = dict()
    maxz = np.max(ref_catalog[ref_catalog['SED_name'] == sed_pick]['z'])
    for z in np.linspace(0, maxz, 3):
        redshifted[z] = ref.getSedData(get_representative(z, sed_pick)['ID'])

    filters = filter_provider.getFilterNames()
    nfilters = len(filters)
    ncols = 2
    nrows = int(np.ceil(nfilters / ncols))

    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(20, 40))
    fig.canvas.layout.height = '20in'
    fig.canvas.layout.width = '100%'
    plt.suptitle(f'{sed_pick}')

    axes = np.ravel(axes)
    for i, filter_name in enumerate(filters):
        ax = axes[i]
        ax.set_title(filter_name)
        transmission = filter_provider.getFilterTransmission(filter_name)
        transmission = transmission[transmission[:,0] > 0.]
        integral = np.trapz(transmission[:,1], transmission[:,0])
        ax.plot(transmission[:,0], transmission[:,1], label=f'Truth ({integral/integral})', linestyle='-')
        for z, sed in redshifted.items():
            sed = truncateSed(sed, [transmission[0,0], transmission[-1,0]])
            interp_filter = np.interp(grid, transmission[:,0], transmission[:,1], left=0, right=0)
            subintergral = np.trapz(interp_filter, grid)
            ax.plot(grid, interp_filter, label=f'{z:.2} ({subintergral/integral:.3f})', linestyle=':')

        ax.legend()

In [ ]:
plot_by_sed(sed_names[8])

In [ ]:
plot_by_sed(sed_names[0])

In [ ]:
sed = ref.getSedData(get_representative(0, sed_names[0])['ID'])

In [ ]:
plt.figure()
plt.hist(np.diff(sed[:,0]), bins=20)
plt.show()